In [22]:
import pystac
from pystac import Link, Asset
from datetime import datetime

In [2]:
from pystac.extensions.label import LabelExtension
from pystac.extensions.label import LabelType
from pystac.extensions.label import LabelClasses
from pystac.extensions.label import LabelStatistics

from pystac.extensions.version import ItemVersionExtension

In [3]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [4]:
aws_url = "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220524_0_L2A"
# labels_geojson_fname = 'labels-v1.geojson'
labels_geojson_fname = 'label-train.geojson'

In [5]:
source_item = pystac.read_file(aws_url)
source_item

<Item id=S2A_10TFK_20220524_0_L2A>

In [6]:
source_item.get_assets()['SCL']

<Asset href=https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2022/5/S2A_10TFK_20220524_0_L2A/SCL.tif>

In [7]:
label_item = pystac.Item(
    id="label_{}".format("lu"),
    geometry=source_item.geometry,
    bbox=source_item.bbox,
    datetime=datetime.utcnow(),
    properties={},
)

In [8]:
label_classes = {
    "NO_DATA": 0,
    "SATURATED_OR_DEFECTIVE": 1,
    "CAST_SHADOWS": 2,
    "CLOUD_SHADOWS": 3,
    "VEGETATION": 4,
    "NOT_VEGETATED": 5,
    "WATER": 6,
    "UNCLASSIFIED": 7,
    "CLOUD_MEDIUM_PROBABILITY": 8,
    "CLOUD_HIGH_PROBABILITY": 9,
    "THIN_CIRRUS": 10,
    "SNOW or ICE": 11,
}

In [9]:
label = LabelExtension.ext(label_item, add_if_missing=True)
label.apply(
    label_description="Land cover labels",
    label_type=LabelType.VECTOR,
    label_tasks=["segmentation", "regression"],
    label_classes=[LabelClasses(label_classes)],
    label_methods=["manual"],
    label_properties=["class"],
)
label.add_source(source_item=source_item)
label.add_geojson_labels(labels_geojson_fname)

In [10]:
version = ItemVersionExtension(label_item)

version.apply(version="0.1", deprecated=False)

label_item.stac_extensions.append(
    "https://stac-extensions.github.io/version/v1.2.0/schema.json"
)

In [11]:
pystac.write_file(label_item, dest_href="label-item.json")

In [12]:
ml_aoi_item = pystac.Item(
    id="ml-aoi",
    geometry=source_item.geometry,
    bbox=source_item.bbox,
    datetime=datetime.utcnow(),
    properties={"ml-aoi:split": "train"},
)

In [13]:
ml_aoi_item.add_link(
    Link(
        rel="derived_from",
        target=aws_url,
        extra_fields={"ml-aoi:role": "feature"},
    )
)

ml_aoi_item.add_link(
    Link(
        rel="derived_from",
        target="label-item.json",
        extra_fields={"role": "ml-aoi:label"},
    )
)

In [14]:
label_asset = Asset(
    href=labels_geojson_fname,
    description="label",
    media_type=pystac.MediaType.GEOJSON,
    roles=["label"],
    extra_fields={"ml-aoi:role": "label"},
)

ml_aoi_item.add_asset(key="label", asset=label_asset)

In [15]:
ml_aoi_item.stac_extensions = [
    "https://stac-extensions.github.io/ml-aoi/v0.1.0/schema.json"
]

In [16]:
ml_item_fname = "item-label-train.json" #"ml-aoi-item.json" 
pystac.write_file(ml_aoi_item, dest_href=ml_item_fname)